In [33]:
import os
import json
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from keras.utils import to_categorical

import matplotlib.pyplot as plt
import eel
from starknet_py.net.full_node_client import FullNodeClient
from starknet_py.net.account.account import Account
from starknet_py.net.models.chains import StarknetChainId
from starknet_py.net.signer.stark_curve_signer import KeyPair
from starknet_py.contract import Contract
from starknet_py.cairo.felt import encode_shortstring
from starknet_py.common import create_sierra_compiled_contract
from starknet_py.common import create_casm_class
from starknet_py.hash.casm_class_hash import compute_casm_class_hash
from starknet_py.net.client_models import ResourceBounds, EstimatedFee, PriceUnit
from pprint import pprint

from typing import List, Optional, Tuple
from common import globalState

import pandas as pd

# import aioconsole
import asyncio

import numpy as np

from basic_conversions import wfloat, wfloat_to_float, to_hex, from_hex

# ----------------------------------------------------------------------

ACCOUNTS_PATH = os.path.join("data", "sepolia.json")

RESSOURCE_BOUND_COMMON = ResourceBounds(59806, 342990458309864)
RESOURCE_BOUND_UPDATE_PREDICTION = RESSOURCE_BOUND_COMMON
RESOURCE_BOUND_UPDATE_PROPOSITION = RESSOURCE_BOUND_COMMON
RESOURCE_BOUND_VOTE_FOR_A_PREDICTION = RESSOURCE_BOUND_COMMON


In [34]:
account = Account(
    client=globalState.client,
    address="0x014C66E352A24Ae28c31d215a0dBf49bc1c349c2a00863Af42F5b56b41f37803",
    key_pair=KeyPair.from_private_key("0x07b9c4b60fc17db5aa92ca508adc83cd83e5a2133110badb7177e3a55963144a"),
    chain=StarknetChainId.SEPOLIA
)

In [35]:
SAMPLE_SIZE = 5
DATASET_PATH = os.path.join("..", "data", "star_classification.csv")


df = pd.read_csv(DATASET_PATH)

X = df.drop(columns=["class"])
scaler = StandardScaler()
X_normalized = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X_normalized = X_normalized.astype(float)
X = X_normalized.to_numpy()

Y = df["class"]
Y = np.array(LabelEncoder().fit_transform(y=Y.values))
Y = np.array(OneHotEncoder().fit_transform(Y.reshape(-1, 1)).toarray()).astype(float)

train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.2, random_state=42)

train_X = np.array(train_X)
train_Y = np.array(train_Y)
test_X = np.array(test_X)
test_Y = np.array(test_Y)

In [36]:
sample = train_X[:5]

In [37]:
contract = await Contract.from_address(
            provider=account, 
            address=globalState.DEPLOYED_ADDRESS
)

In [38]:
inputs_as_felt = [
    [ wfloat(x) for x in line ] for line in sample
]

In [39]:
result = await contract.functions["predict"].invoke_v3(
            inputs=inputs_as_felt, for_storage=True, 
            l1_resource_bounds=RESOURCE_BOUND_UPDATE_PREDICTION
        )

In [44]:
result.invoke_transaction

InvokeV3(version=3, signature=[2324044856980178042615093385415274431713584631949878438276124836202759323824, 3243368416688376737649141283247536364614583956684215900899115734625359730917], nonce=67, resource_bounds=ResourceBoundsMapping(l1_gas=ResourceBounds(max_amount=59806, max_price_per_unit=342990458309864), l2_gas=ResourceBounds(max_amount=0, max_price_per_unit=0)), tip=0, nonce_data_availability_mode=<DAMode.L1: 0>, fee_data_availability_mode=<DAMode.L1: 0>, paymaster_data=[], calldata=[1, 471281052904487870154949072672806528197351462040762969268927468450700519413, 788312007879804315456320911315011427471948705966062620826543392582291438000, 92, 5, 17, 3618502788666131213697322783095070105623107215331596699973092056135871624982, 3618502788666131213697322783095070105623107215331596699973092056135871786583, 780705, 10765, 27393, 467587, 308356, 21403, 3618502788666131213697322783095070105623107215331596699973092056135871624826, 0, 1568078, 36185027886661312136973227830950701056231072